In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import os

# Carregar o dataset
df = pd.read_csv(r'C:\diabetes.csv')
# Tratar valores nulos
df = df.fillna(df.median())

# Separar features e target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Balancear com SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

# Treinar modelo com GridSearchCV
param_grid = {
    'n_estimators': [200, 400],
    'max_depth': [7, 10],               # Reduza a profundidade máxima drasticamente
    'min_samples_split': [15, 20],      # Aumente o mínimo de amostras para dividir
    'min_samples_leaf': [10, 15],       # Aumente o mínimo de amostras por folha
    'max_features': ['sqrt', 'log2']    # Limita o número de features por árvore
}

grid = GridSearchCV( # testa várias combinações de hiperparâmetros do modelo para encontrar a que oferece o melhor desempenho
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,
    scoring='f1'  
)
grid.fit(X_train_bal, y_train_bal)


# Melhor modelo
model = grid.best_estimator_

# Avaliar o modelo
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Acurácia:", acc)
print(classification_report(y_test, y_pred))
print("Melhores parâmetros:", grid.best_params_)

# Salvar o modelo

joblib.dump(model, 'diabetes_model.joblib')
print("Modelo salvo em diabetes_model.joblib")

Acurácia: 0.7662337662337663
              precision    recall  f1-score   support

           0       0.87      0.75      0.80        99
           1       0.64      0.80      0.71        55

    accuracy                           0.77       154
   macro avg       0.75      0.77      0.76       154
weighted avg       0.79      0.77      0.77       154

Melhores parâmetros: {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 15, 'n_estimators': 200}
Modelo salvo em artefatos/diabetes_model.joblib
